In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv("covid_toy.csv")
df.sample(3)

,age,gender,fever,cough,city,has_covid
17,40,Female,98.0,Strong,Delhi,No
33,26,Female,98.0,Mild,Kolkata,No
76,80,Male,100.0,Mild,Bangalore,Yes


In [3]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [4]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop('has_covid', axis=1)
y = df['has_covid']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42)

### 1. Normal Approach

In [7]:
si =SimpleImputer()

X_train_fever = si.fit_transform(X_train[['fever']])

X_test_fever = si.transform(X_test[['fever']])

In [8]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.transform(X_test[['cough']])

In [9]:
ohe = OneHotEncoder(drop='first', sparse_output=False)

X_train_encoded = ohe.fit_transform(X_train[['gender', 'city']])
X_test_encoded = ohe.transform(X_test[['gender', 'city']])

In [10]:
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

In [11]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_encoded, X_train_cough), axis=1)
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_encoded, X_test_cough), axis=1)

X_train_transformed.shape

(80, 7)

### 2. Column Trandformer

In [12]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [13]:
transformer.fit_transform(X_train).shape

(80, 7)

In [14]:
transformer.transform(X_test).shape

(20, 7)